In [ ]:
!unzip -t traqid.zip


Archive:  traqid.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of traqid.zip or
        traqid.zip.zip, and cannot find traqid.zip.ZIP, period.


In [8]:
!pip install ultralytics opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.1 MB/s eta 0:00:00


In [12]:
from ultralytics import YOLO
import cv2

# Load YOLOv8 pretrained model
model = YOLO("yolov8n.pt")

# Vehicle classes in COCO
VEHICLE_CLASSES = ["car", "motorcycle", "bus", "truck"]

# Read image
image = cv2.imread("10.png")

# Run YOLO detection
results = model(image)

vehicle_count = 0

for box in results[0].boxes:
    class_id = int(box.cls[0])
    class_name = model.names[class_id]

    if class_name in VEHICLE_CLASSES:
        vehicle_count += 1

        # Bounding box
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Label
        cv2.putText(
            image,
            class_name,
            (x1, y1 - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

print("Total vehicles detected:", vehicle_count)

# Save output image
cv2.imwrite("output_1.png", image)



0: 384x640 4 cars, 4 buss, 1 truck, 141.3ms
Speed: 3.5ms preprocess, 141.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)
Total vehicles detected: 9


True

ANN MODEL


In [13]:
!pip install pandas scikit-learn tensorflow



In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv("TRAQID.csv")
print(df.head())


            created_at  Image   PM2.5     PM10  Temperature  Humidity  \
0  2022-10-22 18:00:00      1  26.115  41.0112        26.95    43.451   
1  2022-10-22 18:00:00      2  26.823  41.9240        26.96    43.463   
2  2022-10-22 18:00:00      3  26.823  41.3698        26.98    43.566   
3  2022-10-22 18:00:00      4  27.118  41.0438        26.99    43.598   
4  2022-10-22 18:00:00      5  26.882  40.1962        26.96    43.676   

         aqi   Season Day_or_Night aqi_cat  
0  43.525000  Monsoon        Night    Good  
1  44.705000  Monsoon        Night    Good  
2  44.705000  Monsoon        Night    Good  
3  45.196667  Monsoon        Night    Good  
4  44.803333  Monsoon        Night    Good  


In [17]:
df = df.drop(columns=["created_at"])


In [18]:
from sklearn.preprocessing import LabelEncoder

le_season = LabelEncoder()
le_day = LabelEncoder()

df["Season"] = le_season.fit_transform(df["Season"])
df["Day_or_Night"] = le_day.fit_transform(df["Day_or_Night"])


In [19]:
X = df.drop(columns=["aqi", "aqi_cat"])
y = df["aqi"]   # numeric → regression


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation="relu"))
model.add(Dense(1))   # AQI value

model.compile(
    optimizer="adam",
    loss="mse",
    metrics=["mae"]
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,625 (10.25 KB)

 Trainable params: 2,625 (10.25 KB)

 Non-trainable params: 0 (0.00 B)

In [24]:
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=16,
    validation_split=0.2
)


Epoch 1/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 111.7246 - mae: 6.1060 - val_loss: 85.1448 - val_mae: 5.1390
Epoch 2/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - loss: 92.6016 - mae: 5.5258 - val_loss: 78.2781 - val_mae: 4.9479
Epoch 3/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 82.4973 - mae: 5.1550 - val_loss: 71.5215 - val_mae: 4.9076
Epoch 4/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 76.9909 - mae: 4.9032 - val_loss: 69.9377 - val_mae: 4.3857
Epoch 5/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 80.8346 - mae: 4.9494 - val_loss: 65.7354 - val_mae: 4.4955
Epoch 6/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 69.9338 - mae: 4.6493 - val_loss: 58.0726 - val_mae: 4.1698
Epoch 7/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - loss: 63.6000 - mae: 4.3241 - val_loss: 54.8788 - val_mae: 4.0695
Epoch 8/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 57.1648 - mae: 4.1013 - val_loss: 52.2404 - val_mae: 3.8411
Epoch 9/10
1068/1068 ━━

In [25]:
predicted_aqi = model.predict(X_test[:5])
print("Predicted AQI:", predicted_aqi.flatten())
print("Actual AQI:", y_test[:5].values)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted AQI: [      89.55      136.14       357.1      126.55      67.231]
Actual AQI: [     89.643      135.53      355.01      114.49      65.257]


In [26]:
le_aqi = LabelEncoder()
df["aqi_cat_encoded"] = le_aqi.fit_transform(df["aqi_cat"])


In [27]:
X = df.drop(columns=["aqi", "aqi_cat", "aqi_cat_encoded"])
y = df["aqi_cat_encoded"]


In [28]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [29]:
model = Sequential()

model.add(Dense(64, activation="relu", input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation="relu"))
model.add(Dense(len(np.unique(y)), activation="softmax"))

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,790 (10.90 KB)

 Trainable params: 2,790 (10.90 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.2)
loss, acc = model.evaluate(X_test, y_test)
print("Classification Accuracy:", acc)


Epoch 1/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.9141 - loss: 0.2291 - val_accuracy: 0.9435 - val_loss: 0.1621
Epoch 2/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9406 - loss: 0.1590 - val_accuracy: 0.9492 - val_loss: 0.1352
Epoch 3/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9549 - loss: 0.1283 - val_accuracy: 0.9621 - val_loss: 0.1103
Epoch 4/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9591 - loss: 0.1104 - val_accuracy: 0.9583 - val_loss: 0.1011
Epoch 5/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9649 - loss: 0.0967 - val_accuracy: 0.9705 - val_loss: 0.0816
Epoch 6/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9677 - loss: 0.0859 - val_accuracy: 0.9700 - val_loss: 0.0749
Epoch 7/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9701 - loss: 0.0767 - val_accuracy: 0.9665 - val_loss: 0.0797
Epoch 8/10
1068/1068 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9706 - loss: 0.0772 - 